In [1]:
import os
import pandas as pd
import numpy as np
# import validation_data_input
import plotly.express as px
import toml
import math
import psrc_theme

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color" # set plotly template

In [2]:
%store -r validation_data

config = validation_data.config.copy()
tour = validation_data.tour.copy()
trip = validation_data.trip.copy()

In [3]:
# df_person = person.copy()
df_tour = tour.copy()
df_trip = trip.copy()

# ptype_cat = {1: "1: full time worker",
#              2: "2: part time worker",
#              3: "3: non-worker age 65+",
#              4: "4: other non-working adult",
#              5: "5: university student",
#              6: "6: grade school student/child age 16+",
#              7: "7: child age 5-15",
#              8: "8: child age 0-4"}
pdpurp_cat = {1: "Work",
              2: "School",
              3: "Escort",
              4: "Personal Business",
              5: "Shop",
              6: "Meal",
              7: "Social",
              8: "Social",
              9: "Personal Business"}
# df_person['pptyp_label'] = df_person['pptyp'].map(ptype_cat)
# df_person['pptyp_label'] = df_person['pptyp_label'].astype(pd.CategoricalDtype(["1: full time worker","2: part time worker","3: non-worker age 65+","4: other non-working adult","5: university student","6: grade school student/child age 16+","7: child age 5-15","8: child age 0-4"]))

df_tour['pdpurp_label'] = df_tour['pdpurp'].map(pdpurp_cat)
df_tour['pdpurp_label'] = df_tour['pdpurp_label'].astype(pd.CategoricalDtype(["Work","School","Shop","Meal","Escort","Personal Business"]))

df_tour['tlvorg_hr'] = df_tour['tlvorig'].apply(lambda row: int(math.floor(row/60)))


df_trip['dpurp_label'] = df_trip['dpurp'].map(pdpurp_cat)
df_trip['dpurp_label'] = df_trip['dpurp_label'].astype(pd.CategoricalDtype(["Work","School","Shop","Meal","Escort","Personal Business"]))

df_trip['dept_hr'] = df_trip['deptm'].apply(lambda row: int(math.floor(row/60)))

# df_tour = df_tour.merge(df_person, how='left', on=['pno','hhno','source'])

## departure hour

In [4]:
df_plot = df_tour.groupby(['tlvorg_hr','source'])['toexpfac'].agg(sample_count='size', toexpfac='sum').reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['toexpfac']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot, x="tlvorg_hr", y="percentage", color="source",
             barmode="group",title="tour origin departure hour",
             hover_data=['toexpfac','sample_count'])
fig.update_layout(height=400, width=700, font=dict(size=11),
                  xaxis = dict(dtick = 1, categoryorder='category ascending'),
                  yaxis=dict(tickformat=".0%"))
fig.show()

In [5]:
df_plot = df_trip.groupby(['dept_hr','source'])['trexpfac'].agg(sample_count='size', trexpfac='sum').reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['trexpfac']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot, x="dept_hr", y="percentage", color="source",
             barmode="group",title="trip origin departure hour",
             hover_data=['trexpfac','sample_count'])
fig.update_layout(height=400, width=700, font=dict(size=11),
                  xaxis = dict(dtick = 1, categoryorder='category ascending'),
                  yaxis=dict(tickformat=".0%"))
fig.show()

## time choice by purpose

In [6]:
df_plot = df_tour.groupby(['tlvorg_hr','pdpurp_label','source'])['toexpfac'].agg(sample_count='size', toexpfac='sum').reset_index()
df_plot['percentage'] = df_plot.groupby(['pdpurp_label','source'], group_keys=False)['toexpfac']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot.sort_values(['source','pdpurp_label']),
             x="tlvorg_hr", y="percentage", color="source",barmode="group",
             facet_row='pdpurp_label',
             hover_data=['toexpfac','sample_count'],
             title="tour origin departure hour by purpose")
fig.update_layout(height=1500, width=850)
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.for_each_xaxis(lambda a: a.update(dict(dtick = 1, categoryorder='category ascending')))
fig.for_each_yaxis(lambda a: a.update(dict(tickformat=".0%")))
fig.show()

In [7]:
df_plot = df_trip.groupby(['dept_hr','dpurp_label','source'])['trexpfac'].agg(sample_count='size', trexpfac='sum').reset_index()
df_plot['percentage'] = df_plot.groupby(['dpurp_label','source'], group_keys=False)['trexpfac']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot.sort_values(['source','dpurp_label']),
             x="dept_hr", y="percentage", color="source",barmode="group",
             facet_row='dpurp_label',
             hover_data=['trexpfac','sample_count'],
             title="trip origin departure hour by purpose")
fig.update_layout(height=1500, width=850)
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.for_each_xaxis(lambda a: a.update(dict(dtick = 1, categoryorder='category ascending')))
fig.for_each_yaxis(lambda a: a.update(dict(tickformat=".0%")))
fig.show()